<a href="https://colab.research.google.com/github/Javier-Perales/CatVsDog/blob/main/DvC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Del modelo a la aplicación
En este sprint vamos a construir una aplicación real de Machine Learning en Python para resolver un problema de clasificación de imágenes "Perro/Gato".

En esta primera aproximación **low-code**, usaremos el modelo entrenado en Teachable Machine y construiremos la aplicación sobre Google Colab. Nos centraremos en la inferencia haciendo uso de **TensorFlow/Keras** para cargar el modelo y hacer predicciones.

## Cargar el modelo

Previamente a exportar el modelo entrenado en Teachable Machine a Python, debemos tener claro algunos conceptos clave. Las imágenes utilizadas para entrenar cada una de las categorías puedes descargarlas de ![Cats and Dogs light](https://zenodo.org/records/5226945).

El repositorio contiene imágenes (500 gatos y 500 perros) y 400 imágenes para el conjunto de prueba (200 cada uno).

Una vez entrenado el modelo, lo exportamos en formato **Keras**.

👀: Google Colab ha actualizado la librería Keras a la versión 3.0, mientras que Teachable Machine exporta los modelos en formato Keras 2.x. Por tanto, debemos indicar el siguiente parámetro para garantizar la compatibilidad.

In [ ]:
import os
# 1. Activamos el modo legacy (Keras 2)
os.environ['TF_USE_LEGACY_KERAS'] = '1'

El parámetro `compile=False` se usa para evitar que Keras intente compilar el modelo al cargarlo, lo cual no es necesario para la inferencia.

In [ ]:
#from tf_keras.models import load_model
from tensorflow import keras
try:
    mi_modelo = keras.models.load_model("/content/drive/MyDrive/DogVsCat/modelo/keras_model.h5", compile=False)
    print("✅ ¡Éxito! Modelo .h5 cargado con el paquete tf-keras.")
    model.summary()
except Exception as e:
    print(f"❌ Error al cargar el modelo: {e}")

✅ ¡Éxito! Modelo .h5 cargado con el paquete tf-keras.
❌ Error al cargar el modelo: name 'model' is not defined


In [ ]:
# Carga las etiquetas de las clases
nombre_clases = open("/content/drive/MyDrive/DogVsCat/modelo/labels.txt", "r").readlines()

print(nombre_clases)

['0 gatos\n', '1 Perros\n']


## Carga de imágenes

En esta primera aproximación, se cargará la imagen que quieras clasificar a Google Colab.

Para cargar imágenes en Python, usaremos la librería PIL (Python Imaging Library), en concreto la clase Image e ImageOps ('from PIL import Image, ImageOps').

In [ ]:
from PIL import Image, ImageOps

imagen= Image.open("/content/drive/MyDrive/DogVsCat/cats_dogs_light/cats_dogs_light/test/cat.9818.jpg").convert("RGB")

La imagen DEBE transformarse para ser idéntica a las de entrenamiento.

Esto incluye:

* Cambiar tamaño de la imagen para que tenga al menos 224x224 píxeles y luego se recorta desde el centro.
* Normalizar los valores de los píxeles (ej. de [0, 255] a [-1, 1]). Para ello deberemos utilizar la librería Numpy (`import numpy as np`)
* Por último. para desplazar el valor entre -1 y +1, le restaremos -1 a cada valor de la celda.



In [ ]:
import numpy as np
size = (224, 224)
imagen = ImageOps.fit(imagen, size, Image.Resampling.LANCZOS)

imagen_array = np.asarray(imagen)

normalizada_imagen_array = (imagen_array.astype(np.float32) / 127.5) - 1


Las IAs no procesan imágenes "una a una", sino en lotes. Aunque solo tengamos una foto, debemos meterla en un array de 1 hueco. Ese hueco, a su vez, debe contener la imagen pre-procesada.

Para clasificar la imagen preprocesada, usamos el método predict() del modelo cargado en la Fase 1 ('mi_modelo').

predict está diseñado para procesar conjuntos de datos a la vez. Por eso, necesita recibir una lista (un lote), aunque esa lista solo tenga un elemento.

In [ ]:
lote_imagenes = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Load the image into the array
lote_imagenes[0] = normalizada_imagen_array

# Predicts the model
resultados = mi_modelo.predict(lote_imagenes)
print(resultados)

indice = np.argmax(resultados[0])
print(indice)
etiqueta = nombre_clases[indice]
probabilidad = resultados[0][indice]

# Print prediction and confidence score
print("Clase:", etiqueta[2:])
print("Probabilidad:", probabilidad)

1/1 [==============================] - 0s 32ms/step
[[0.9980236  0.00197635]]
0
Clase: gatos

Probabilidad: 0.9980236
